In [1]:
import pandas as pd 
import numpy as np 
import json
import re
from functools import reduce

In [91]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer,MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
from xgboost import XGBClassifier

In [4]:
from lightgbm import LGBMClassifier

In [5]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [6]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")



In [7]:
train.head()

,ID,Prediction
0,user0x410,1
1,user0x432,1
2,user0x16a3,1
3,user0x1ad1,1
4,user0x174d,0


In [8]:
def get_temp(text):
    #print(text)
    temp =  re.findall("\d+\.\d+",text)
    if len(temp)> 0:
        return temp[0]
    else:
        return None

In [9]:
def get_data(filename):
    b = json.load(open("dataset/trainConversations/"+filename+".json"))
    if 'What Symptoms do you have? ' == list(b.keys())[3]:
        text = list(b.values())[0] + list(b.keys())[1] + list(b.values())[1]
        temp =  get_temp(list(b.values())[2])
        symp = None
        tr_type = list(b.values())[4]
        loc = None
    elif "Share your Current location? " not in b:
        text = list(b.values())[0] + list(b.keys())[1]
        temp =  get_temp(list(b.keys())[2])
        symp = list(b.keys())[3]
        tr_type = list(b.keys())[4]
        loc = None
    else:
        text = list(b.values())[0] 
        temp =  get_temp(list(b.values())[1])
        symp = list(b.values())[2]
        tr_type = list(b.values())[3]
        loc = re.findall("\d+\.\d+",list(b.values())[4])

        
    return pd.Series([text,temp,symp,tr_type,loc])

In [10]:
def get_data1(filename):
    b = json.load(open("dataset/testConversations/"+filename+".json"))
    if 'What Symptoms do you have? ' == list(b.keys())[3]:
        text = list(b.values())[0] + list(b.keys())[1] + list(b.values())[1]
        temp =  get_temp(list(b.values())[2])
        symp = None
        tr_type = list(b.values())[4]
        loc = None
    elif "Share your Current location? " not in b:
        text = list(b.values())[0] + list(b.keys())[1]
        temp =  get_temp(list(b.keys())[2])
        symp = list(b.keys())[3]
        tr_type = list(b.keys())[4]
        loc = None
    else:
        text = list(b.values())[0] 
        temp =  get_temp(list(b.values())[1])
        symp = list(b.values())[2]
        tr_type = list(b.values())[3]
        loc = re.findall("\d+\.\d+",list(b.values())[4])

        
    return pd.Series([text,temp,symp,tr_type,loc])

In [11]:
train[['text','temp','symp','tr_type','location']] = train.ID.apply(get_data)
test[['text','temp','symp','tr_type','location']] = test.ID.apply(get_data1)

In [12]:
train.tail()

,ID,Prediction,text,temp,symp,tr_type,location
4895,user0x389,0,What started the virus? Search for Covid19 cau...,98.42,Nothing,type_2,"[13.73, 93.87]"
4896,user0x1448,0,Is there a flu vaccine for the virus How did c...,96.72,Nothing,type_1,"[17.62, 76.96]"
4897,user0xf8c,0,I am wondering do I have Corona if my body hur...,98.05,Nothing,type_1,"[14.84, 78.09]"
4898,user0xeb,1,Do you know if fevers are symptoms of Covid-19...,99.72,loss_of_smell,type_2,"[20.4, 91.62]"
4899,user0x1425,0,Can you show me how is the virus spreading? Wh...,97.36,None,type_1,None


In [13]:
test.tail()

,ID,text,temp,symp,tr_type,location
2095,user0x1855,Is there a treatment for the symptoms of covid...,98.17,tiredness,type_3,"[18.73, 72.45]"
2096,user0x17a8,I feel more tired than usual. What is the orig...,101.91,"tiredness', 'sore_throat",type_2,"[19.84, 90.49]"
2097,user0xac2,What do i need to know about the emergence of ...,100.74,loss_of_taste,type_3,"[10.96, 91.62]"
2098,user0x1648,Can I take aspirin if I have corona virus how ...,106.17,"loss_of_taste', 'tiredness",type_1,"[14.84, 90.49]"
2099,user0x62b,Is it safe to use ibuprofen if I have corona v...,96.92,loss_of_taste,type_3,"[13.73, 91.62]"


In [14]:
train.temp = train.temp.astype('float')
test.temp = test.temp.astype('float')

In [15]:
train.shape

(4900, 7)

In [25]:
train.isna().sum()

ID             0
Prediction     0
text           0
temp           0
symp          60
tr_type        0
location      67
dtype: int64

In [17]:
test.isna().sum()

ID           0
text         0
temp         1
symp        33
tr_type      0
location    40
dtype: int64

In [18]:
def clean_text(text):
    if text:
        return re.sub(r"[^a-z ,]","",text.lower())
    return None

In [19]:
train.symp = train.symp.apply(clean_text)
test.symp  = test.symp.apply(clean_text)

In [111]:
# train.symp.fillna("",inplace=True)
# test.symp.fillna("",inplace=True)

### Predict Symptoms

In [20]:
classif = OneVsRestClassifier(SVC(kernel='linear'))

In [30]:
mlb = MultiLabelBinarizer()

In [39]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z. ]","",text)
    text_wsw = [word for word in text.split() if word not in stop_words]
    text_lem = [lemmatizer.lemmatize(word) for word in text_wsw]
    return " ".join(text_lem)

In [47]:
train['clean_text'] = train.text.apply(preprocess_text)

In [48]:
test['clean_text'] = test.text.apply(preprocess_text)

In [83]:
X = train[~train.symp.isna()].clean_text
Xt = test[~test.symp.isna()].clean_text
Xp = train[train.symp.isna()].clean_text

y = train[~train.symp.isna()].symp
yt = test[~test.symp.isna()].symp

In [84]:
tfidf = TfidfVectorizer(min_df=5,max_df=.9)

In [85]:
y = mlb.fit_transform(y)

In [86]:
yt= mlb.transform(yt)

In [77]:
Xvec = tfidf.fit_transform(X).toarray()
Xtvec = tfidf.transform(Xt).toarray()

In [78]:
Xvec.shape

(4840, 1133)

In [93]:
cl_pipe = Pipeline(steps=[('pca',PCA()),
                         ('model',OneVsRestClassifier(RandomForestClassifier()))
                         ])

In [95]:
xtrain,xtest,ytrain,ytest = train_test_split(Xvec,y,test_size=.3)

In [96]:
cl_pipe.fit(xtrain,ytrain)

Pipeline(steps=[('pca', PCA()),
                ('model',
                 OneVsRestClassifier(estimator=RandomForestClassifier()))])

In [97]:
yt_hat=cl_pipe.predict(xtest)

In [98]:
metrics.accuracy_score(ytest,yt_hat)

0.0006887052341597796

In [101]:
train[train.symp.isna()]

,ID,Prediction,text,temp,symp,tr_type,location,clean_text
28,user0x1dd,0,What is not true about the virus Covid19 and i...,96.07,None,type_1,None,true virus covid allergic effect post month se...
95,user0x13b9,1,Tell me if coughing is a symptom of coronaviru...,99.86,None,type_3,None,tell coughing symptom coronavirus. know diarrh...
181,user0x2a8,0,I feel dizzy. Where started corona virus? Wher...,95.67,None,type_1,None,feel dizzy. started corona virus virus spreadi...
216,user0x93b,0,Corona and breathlessness.\rCorona asthma inha...,94.76,None,type_2,None,corona breathlessness.corona asthma inhaler.co...
270,user0x1731,0,Cornavirus pandemic. is coronavirus real how m...,94.90,None,type_3,None,cornavirus pandemic. coronavirus real manny di...
288,user0x1574,0,Do you think I have the Corona virus because I...,94.44,None,type_2,None,think corona virus cough think corona virus co...
569,user0xa13,0,Which remedies can I take to fight the virus? ...,94.23,None,type_2,None,remedy take fight virus corona general hygiene...
995,user0x3de,0,Corona and its precautions. What are the sympt...,97.58,None,type_1,None,corona precautions. symptom coronavirus like l...
1041,user0x1413,0,was coronavirus manufactured how did corona vi...,95.73,None,type_3,None,coronavirus manufactured corona virus develop ...
1044,user0x1366,1,is the corona virus in st louis missouri Coron...,104.71,None,type_1,None,corona virus st louis missouri corona breathle...


In [91]:
tfidf = TfidfVectorizer()
cnt = CountVectorizer()

In [92]:
symp_vec = cnt.fit_transform(train.symp).toarray()
test_symp_vec = cnt.transform(test.symp).toarray()

In [93]:
train_vec = np.concatenate([symp_vec,np.array(train.temp.values.reshape(-1,1))],axis=1)

In [94]:
test_vec = np.concatenate([test_symp_vec,np.array(test.temp.values.reshape(-1,1))],axis=1)

In [95]:
mdl = RandomForestClassifier(class_weight="balanced",n_estimators=500)

In [96]:
xgb = XGBClassifier()
lgb= LGBMClassifier()

In [97]:
xgb_pipe = Pipeline(steps=[('scaler',StandardScaler()),
                           ('mdl',xgb)])

In [98]:
cross_val_score(xgb_pipe,train_vec,train.Prediction,cv=3,scoring='f1')

array([0.96075778, 0.96098563, 0.95759234])

In [53]:
xgb_pipe.fit(train_vec,train.Prediction)

Pipeline(steps=[('scaler', StandardScaler()),
                ('mdl',
                 XGBClassifier(base_score=0.5, booster=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints=None,
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=0, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method=None,
                               validate_parameters=False, verbosity=None))])

In [59]:
yhat  = xgb_pipe.predict(test_vec)

In [60]:
res = pd.DataFrame()

In [61]:
res['ID'] = test.ID
res['Prediction'] = yhat

In [62]:
res.to_csv("XGBoost_base.csv",index=False)